# TECHIN 509: Melody Generator with Files & Testing

In this assignment, you will:
* Save and load melodies from files so your work is reproducible.
* Write simple tests to check that your functions behave correctly.
* Strengthen habits of writing **reliable, reusable code**.

#### Part 1 — Work with Files
**Load dataset from a file**

In [ ]:
import random
from collections import defaultdict, Counter


def load_melodies(path: str) -> list[list[tuple[str, float]]]:
    """Load melodies from a file, returning a list of (note, duration) tuples"""
    melodies = []
    try:
        with open(path, 'r') as f:
            for line in f:
                notes = []
                for token in line.strip().split():
                    if "_" in token:
                        pitch, duration = token.split("_")
                        notes.append((pitch, float(duration)))
                    else:
                        notes.append((token, 1.0))  # default duration
                melodies.append(notes)
    except FileNotFoundError:
        print(f"File not found: {path}")
        print("Please make sure the dataset file exists.")
    return melodies

def save_melodies(melodies: list[list[tuple[str, float]]], path: str) -> None:
    """Save melodies to a file, one melody per line"""
    with open(path, 'w') as f:
        for melody in melodies:
            line = " ".join(f"{note}_{duration}" for note, duration in melody)
            f.write(line + "\n")

print("Testing load_melodies function:")
melodies = load_melodies('data/melody.txt')
print(f"Loaded {len(melodies)} melodies")

if melodies:
    print(f"First melody has {len(melodies[0])} notes")
    print(f"First 5 notes: {melodies[0][:5]}")

if melodies:
    save_melodies(melodies[:2], 'data/test_save.txt')  # save first 2 melodies


Testing load_melodies function:
Loaded 9 melodies
First melody has 3 notes
First 5 notes: [('E4', 0.25), ('E4', 0.25), ('E4', 0.5)]


**Save generated melodies to a file**

In [9]:
def save_melodies(melodies: list[list[str]], path: str) -> None:
    """Save melodies (note strings) to a file, one melody per line"""
    try:
        with open(path, 'w') as file:
            for melody in melodies:
                line = " ".join(melody)  # no unpacking
                file.write(line + "\n")
        print(f"Saved {len(melodies)} melodies to {path}")
    except Exception as e:
        print(f"Error saving file: {e}")

test_melodies = [
    ['C4_0.25', 'D4_0.25', 'E4_0.25', 'G4_0.5'],
    ['F3_0.5', 'A3_0.25', 'C4_0.25'],
    ['G3_0.25', 'B3_0.25', 'D4_0.5', 'F4_0.25']
]

save_melodies(test_melodies, 'output/test_output.txt')


Saved 3 melodies to output/test_output.txt


**Robustness**

In [13]:
def load_melodies(path: str) -> list[list[str]]:
    """
    Read melodies from a file and return as list of note strings.
    
    Args:
        path: Path to the file containing melodies
        
    Returns:
        List of melodies, where each melody is a list of note strings
    """
    try:
        with open(path, 'r') as file:
            melodies = []
            for line in file:
                line = line.strip()  # remove whitespace
                if line:  # skip empty lines
                    notes = line.split()  # split by spaces
                    melodies.append(notes)
            return melodies

    except FileNotFoundError:
        print(f"File not found: {path}")
        print("Please make sure the dataset file exists.")
        return []

    except Exception as e:
        print(f"Error reading file: {e}")
        return []

# Test the robust function

print("Testing robust load_melodies function:")
melodies = load_melodies('data/melody.txt')
print(f"Loaded {len(melodies)} melodies")

if melodies:
    print(f"First melody has {len(melodies[0])} notes")
    print(f"First 5 notes: {melodies[0][:5]}")


Testing robust load_melodies function:
Loaded 8 melodies
First melody has 3 notes
First 5 notes: ['E4_0.25', 'E4_0.25', 'E4_0.5']


#### Part 2 - Add Testing

In [2]:
import unittest
import tempfile
import os

# Functions to test
def load_melodies(file_path: str) -> list[list[str]]:
    """Load melodies from a file, returning a list of melodies (each a list of note strings)."""
    try:
        with open(file_path, 'r') as f:
            melodies = [line.strip().split() for line in f if line.strip()]
        return melodies
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def save_melodies(melodies: list[list[str]], file_path: str) -> None:
    """Save melodies to a file, one melody per line."""
    try:
        with open(file_path, 'w') as f:
            for melody in melodies:
                f.write(' '.join(melody) + '\n')
    except Exception as e:
        print(f"Error saving file: {e}")

def parse_note_str(note_str: str) -> tuple[str, float]:
    """Parse a note string like 'C4_0.25' into (note, duration)."""
    parts = note_str.split('_')
    if len(parts) != 2:
        raise ValueError(f"Invalid note format: {note_str}")
    note, dur = parts[0], float(parts[1])
    return note, dur


# Unit tests
class TestMelodyFunctions(unittest.TestCase):

    def test_load_valid_file(self):
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
            f.write("C4_0.25 D4_0.25 E4_0.5\n")
            f.write("G3_0.5 A3_0.25\n")
            path = f.name
        try:
            melodies = load_melodies(path)
            self.assertEqual(len(melodies), 2)
            self.assertEqual(melodies[0], ["C4_0.25", "D4_0.25", "E4_0.5"])
            self.assertEqual(melodies[1], ["G3_0.5", "A3_0.25"])
        finally:
            os.unlink(path)

    def test_load_missing_file(self):
        result = load_melodies("no_such_file.txt")
        self.assertEqual(result, [])

    def test_load_skips_empty_lines(self):
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
            f.write("C4_0.25 D4_0.25\n")
            f.write("\n")
            f.write("E4_0.5\n")
            path = f.name
        try:
            melodies = load_melodies(path)
            self.assertEqual(len(melodies), 2)
            self.assertEqual(melodies[0], ["C4_0.25", "D4_0.25"])
            self.assertEqual(melodies[1], ["E4_0.5"])
        finally:
            os.unlink(path)

    def test_save_melodies_creates_file(self):
        melodies = [["C4_0.25", "D4_0.25"], ["G3_0.5"]]
        with tempfile.NamedTemporaryFile(delete=False) as f:
            path = f.name
        try:
            save_melodies(melodies, path)
            with open(path, 'r') as f:
                lines = f.read().splitlines()
            self.assertEqual(lines, ["C4_0.25 D4_0.25", "G3_0.5"])
        finally:
            os.unlink(path)

    def test_parse_note_valid_and_invalid(self):
        note, dur = parse_note_str("F#4_0.25")
        self.assertEqual(note, "F#4")
        self.assertAlmostEqual(dur, 0.25)

        note, dur = parse_note_str("A3_0.5")
        self.assertEqual(note, "A3")
        self.assertAlmostEqual(dur, 0.5)

        with self.assertRaises(ValueError):
            parse_note_str("C4")  # Missing duration
        with self.assertRaises(ValueError):
            parse_note_str("InvalidNoteString")

# Run tests
if __name__ == "__main__":
    unittest.main(argv=[''], exit=False, verbosity=2)




test_load_missing_file (__main__.TestMelodyFunctions.test_load_missing_file) ... ok
test_load_skips_empty_lines (__main__.TestMelodyFunctions.test_load_skips_empty_lines) ... ok
test_load_valid_file (__main__.TestMelodyFunctions.test_load_valid_file) ... ok
test_parse_note_valid_and_invalid (__main__.TestMelodyFunctions.test_parse_note_valid_and_invalid) ... ok
test_save_melodies_creates_file (__main__.TestMelodyFunctions.test_save_melodies_creates_file) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


File not found: no_such_file.txt


#### Part 3 - Playing the Music

In [9]:

import random
from collections import defaultdict, Counter
import os
from midiutil import MIDIFile
import pygame
import time

# File I/O Functions

def load_melodies(path: str) -> list[list[str]]:
    """Load melodies from a file."""
    try:
        with open(path, 'r') as file:
            melodies = []
            for line in file:
                line = line.strip()
                if line:
                    notes = line.split()
                    melodies.append(notes)
            return melodies
    except FileNotFoundError:
        print(f" File not found: {path}")
        return []


def save_melodies(melodies: list[list[str]], path: str) -> None:
    """Save melodies to a file."""
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as file:
            for melody in melodies:
                line = " ".join(melody)
                file.write(line + "\n")
        print(f"✓ Saved {len(melodies)} melodies to {path}")
    except Exception as e:
        print(f" Error saving file: {e}")


# Music Generation Functions

def build_bigram_model(melodies: list[list[str]]) -> dict:
    """Build a bigram model from melodies."""
    bigram = defaultdict(Counter)
    for melody in melodies:
        melody_with_tokens = ["^"] + melody + ["$"]
        for i in range(len(melody_with_tokens) - 1):
            current_note = melody_with_tokens[i]
            next_note = melody_with_tokens[i + 1]
            current_pitch = current_note.split('_')[0] if '_' in current_note else current_note
            next_pitch = next_note.split('_')[0] if '_' in next_note else next_note
            bigram[current_pitch][next_pitch] += 1
    return bigram


def choose_next_note(current_note: str, bigram: dict) -> str:
    """Choose the next note based on bigram probabilities."""
    if current_note not in bigram or len(bigram[current_note]) == 0:
        return "$"
    next_notes = list(bigram[current_note].keys())
    weights = list(bigram[current_note].values())
    return random.choices(next_notes, weights=weights, k=1)[0]


def generate_melody(bigram: dict, max_length: int = 10, start_note: str = None) -> list[str]:
    """Generate a new melody using the bigram model."""
    melody = []
    current_note = start_note if start_note else "^"
    
    while len(melody) < max_length:
        next_note = choose_next_note(current_note, bigram)
        if next_note == "$":
            break
        durations = [0.25, 0.5, 1.0, 2.0]
        duration = random.choice(durations)
        melody.append(f"{next_note}_{duration}")
        current_note = next_note
    
    return melody

# MIDI Conversion Functions 

def note_to_midi_number(note_str: str) -> int:
    """
    Convert a note string like 'C4' or 'F#5' to MIDI note number.
    C4 = 60 (middle C)
    """
    note_map = {
        'C': 0, 'C#': 1, 'Db': 1,
        'D': 2, 'D#': 3, 'Eb': 3,
        'E': 4,
        'F': 5, 'F#': 6, 'Gb': 6,
        'G': 7, 'G#': 8, 'Ab': 8,
        'A': 9, 'A#': 10, 'Bb': 10,
        'B': 11
    }
    
    # Parse note (e.g., "C#4" -> "C#" and "4")
    if len(note_str) >= 2:
        if note_str[1] in ['#', 'b']:
            note = note_str[:2]
            octave = int(note_str[2:])
        else:
            note = note_str[0]
            octave = int(note_str[1:])
        
        # Calculate MIDI number: (octave + 1) * 12 + note_offset
        midi_number = (octave + 1) * 12 + note_map.get(note, 0)
        return midi_number
    
    return 60  # Default to middle C


def melody_to_midi(melody: list[str], output_path: str, tempo: int = 120) -> None:
    """
    Convert a melody to a MIDI file.
    
    Args:
        melody: List of note strings like ['C4_0.5', 'D4_1.0']
        output_path: Path to save the MIDI file
        tempo: BPM (beats per minute)
    """
    # Create MIDI file with 1 track
    midi = MIDIFile(1)
    track = 0
    channel = 0
    volume = 100
    
    # Add track name and tempo
    midi.addTrackName(track, 0, "Generated Melody")
    midi.addTempo(track, 0, tempo)
    
    # Add notes
    time_offset = 0
    for note_str in melody:
        if '_' in note_str:
            pitch_str, duration_str = note_str.split('_')
            duration = float(duration_str)
        else:
            pitch_str = note_str
            duration = 1.0
        
        # Convert to MIDI note number
        pitch = note_to_midi_number(pitch_str)
        
        # Add note to MIDI file
        midi.addNote(track, channel, pitch, time_offset, duration, volume)
        time_offset += duration
    
    # Write to file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'wb') as output_file:
        midi.writeFile(output_file)
    
    print(f"✓ Saved MIDI file: {output_path}")


def play_midi(midi_path: str) -> None:
    """
    Play a MIDI file using pygame on MacBook.
    
    Args:
        midi_path: Path to the MIDI file
    """
    try:
        # Initialize pygame mixer
        pygame.mixer.init()
        
        # Load and play the MIDI file
        pygame.mixer.music.load(midi_path)
        pygame.mixer.music.play()
        
        print(f"🔊 Playing: {midi_path}")
        print("   (Press Ctrl+C to stop)")
        
        # Wait for the music to finish
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
        
        print("✓ Playback finished")
        
    except KeyboardInterrupt:
        pygame.mixer.music.stop()
        print("\n  Playback stopped")
    except Exception as e:
        print(f" Error playing MIDI: {e}")
    finally:
        pygame.mixer.quit()


# Complete Pipeline with Playback

def generate_and_play_music(input_path: str, output_dir: str = 'output',
                            num_melodies: int = 3, melody_length: int = 8,
                            tempo: int = 120, auto_play: bool = True):
    """
    Complete pipeline: load, train, generate, save as MIDI, and play.
    
    Args:
        input_path: Path to training data
        output_dir: Directory for output files
        num_melodies: Number of melodies to generate
        melody_length: Length of each melody
        tempo: BPM for playback
        auto_play: Whether to automatically play generated melodies
    """
    print("\n" + "=" * 60)
    print(" MELODY GENERATION & PLAYBACK PIPELINE")
    print("=" * 60)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Load and train
    print("\n[1/4] Loading training data...")
    melodies = load_melodies(input_path)
    if not melodies:
        return
    print(f"✓ Loaded {len(melodies)} training melodies")
    
    print("\n[2/4] Building bigram model...")
    bigram = build_bigram_model(melodies)
    print(f"✓ Model trained")
    
    # Generate melodies
    print(f"\n[3/4] Generating {num_melodies} melodies...")
    generated = []
    midi_files = []
    
    for i in range(num_melodies):
        melody = generate_melody(bigram, max_length=melody_length)
        generated.append(melody)
        
        print(f"\n   🎼 Melody {i+1}: {' '.join(melody)}")
        
        # Convert to MIDI
        midi_path = f"{output_dir}/melody_{i+1}.mid"
        melody_to_midi(melody, midi_path, tempo=tempo)
        midi_files.append(midi_path)
    
    # Save as text too
    text_path = f"{output_dir}/generated_melodies.txt"
    save_melodies(generated, text_path)
    
    # Play melodies
    if auto_play:
        print(f"\n[4/4] Playing generated melodies...")
        for i, midi_path in enumerate(midi_files):
            print(f"\n  Playing Melody {i+1}/{num_melodies}")
            play_midi(midi_path)
            if i < len(midi_files) - 1:
                print("   Waiting 1 second before next melody...")
                time.sleep(1)
    
    print("\n" + "=" * 60)
    print("PIPELINE COMPLETE!")
    print(f"MIDI files saved in: {output_dir}/")
    print("=" * 60)
    
    return generated, midi_files


# Generate and play music
melodies, midi_files = generate_and_play_music(
    input_path='data/melody.txt',
    output_dir='output',
    num_melodies=3,
    melody_length=8,
    tempo=120,
    auto_play=True
)


 MELODY GENERATION & PLAYBACK PIPELINE

[1/4] Loading training data...
✓ Loaded 8 training melodies

[2/4] Building bigram model...
✓ Model trained

[3/4] Generating 3 melodies...

   🎼 Melody 1: F4_0.5 F4_0.25 F4_0.25 F4_2.0 E4_1.0 D4_0.25 D4_0.25 E4_2.0
✓ Saved MIDI file: output/melody_1.mid

   🎼 Melody 2: E4_0.5
✓ Saved MIDI file: output/melody_2.mid

   🎼 Melody 3: F4_1.0 F4_2.0 F4_2.0 E4_0.5
✓ Saved MIDI file: output/melody_3.mid
✓ Saved 3 melodies to output/generated_melodies.txt

[4/4] Playing generated melodies...

  Playing Melody 1/3
🔊 Playing: output/melody_1.mid
   (Press Ctrl+C to stop)
✓ Playback finished
   Waiting 1 second before next melody...

  Playing Melody 2/3
🔊 Playing: output/melody_2.mid
   (Press Ctrl+C to stop)
✓ Playback finished
   Waiting 1 second before next melody...

  Playing Melody 3/3
🔊 Playing: output/melody_3.mid
   (Press Ctrl+C to stop)
✓ Playback finished

PIPELINE COMPLETE!
MIDI files saved in: output/
